# IAU Projekt 2. fáza

### Autori: Martin Raffáč (50%), Tibor Vanek (50%)

dataset číslo **003**

In [3]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree, metrics
from pandas import read_csv
import seaborn as sns
from matplotlib import pyplot as plt
from scipy import stats
from sklearn.model_selection import RandomizedSearchCV
from matplotlib import pyplot
import os
#import graphviz 
#import pydot
from IPython.display import Image
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import validation_curve
from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.svm import SVR
from sklearn.feature_selection import VarianceThreshold

In [4]:

train_data = pd.read_csv("num_data.csv", sep=',')

Náčítanie trénovacích dát.

In [16]:
data_for_oneR = train_data.copy()

1. Jednoduchý klasifikátor na základe závislosti v dátach 

In [46]:
def oneR(column):

    split = (data_for_oneR[column][data_for_oneR['warning'] == 0].mean() + data_for_oneR[column][data_for_oneR['warning'] == 1].mean())/2
    group = data_for_oneR[data_for_oneR[column] > split]
    group = group.groupby('warning').size()

    if group[0] > group[1]:
        left = 1
        right = 0
    else:
        left = 0
        right = 1
        
    predict = {"split": split, "left": left, "right": right}
    
    data_for_oneR.loc[data_for_oneR[column] < predict["split"], 'helper'] = predict["left"]
    data_for_oneR.loc[data_for_oneR[column] >= predict["split"], 'helper'] = predict["right"]
    accuracy = accuracy_score(data_for_oneR['warning'], data_for_oneR['helper'])
    precision = precision_score(data_for_oneR['warning'], data_for_oneR['helper'])
    recall = recall_score(data_for_oneR['warning'], data_for_oneR['helper'])
    total=(accuracy + precision + recall) / 3
    data_for_oneR.drop('helper', axis=1, inplace=True)

    return {"accuracy": accuracy, "precision": precision, "recall": recall ,"total": total}

In [47]:
accuracy = 0 
columns =  data_for_oneR.columns

for i in columns:
    if i == 'warning':
        continue

    data_oneR = oneR(i)
    
    if data_oneR["total"] > accuracy:
        best = i
        accuracy = data_oneR["total"]
        best_column = data_oneR
        
print('best oneR: ' + best)
print('total_score: ' + str(best_column["total"]))
print('accuracy_score: ' + str(best_column["accuracy"]))
print('precision_score: ' + str(best_column["precision"]))
print('recall_score: ' + str(best_column["recall"]))

best oneR: PAHs
total_score: 0.8234555012157715
accuracy_score: 0.8037125988191735
precision_score: 0.857682841068917
recall_score: 0.8089710637592239


Podľa zadania sme zhotovili jednoduchý algoritmus oneR, ktorý sme vložili do funkcie. Následne sme ho vyskúšali na trénovacom datasete, predpokladali sme, že najlepší výsledok dosiahne atrihút PAHs, pretože má najviššiu koreláciu s atribútom warning. Ako je možné vidieť vyššie naše predpoklady sa naplinili.  